In [ ]:
from PIL import Image
import numpy as np
from IPython.display import display
import random

In [ ]:
image_path = 'bulk-carrier.jpg'
num_rows, num_cols = 78, 100

image = Image.open(image_path)
img_array = np.array(image)

def split_into_tiles(img_array, num_rows, num_cols):
    """Split image array into tiles (2D list)."""
    h, w = img_array.shape[:2]
    tile_h = h // num_rows
    tile_w = w // num_cols
    tiles = []
    for i in range(num_rows):
        row_tiles = []
        for j in range(num_cols):
            y0 = i * tile_h
            y1 = (i + 1) * tile_h if i < num_rows - 1 else h
            x0 = j * tile_w
            x1 = (j + 1) * tile_w if j < num_cols - 1 else w
            row_tiles.append(img_array[y0:y1, x0:x1])
        tiles.append(row_tiles)
    return tiles, tile_h, tile_w

tiles, tile_h, tile_w = split_into_tiles(img_array, num_rows, num_cols)

for i in range(num_rows):
    for j in range(num_cols):
        if (j % 2) == 1:
            tiles[i][j] = np.rot90(tiles[i][j], 2)

display(image)


In [ ]:
random_indices = random.sample(
    [(i, j) for i in range(num_rows) for j in range(num_cols)],
    3
)

for row, col in random_indices:
    tile_img = Image.fromarray(tiles[row][col])
    display(tile_img)

In [ ]:
def reconstruct_from_tile_rows(tile_rows):
    """Reconstruct an image from selected tile rows (2D list of tiles)."""
    return np.concatenate([np.concatenate(row, axis=1) for row in tile_rows], axis=0)

odd_tile_rows = [tiles[i] for i in range(0, num_rows, 2)]
even_tile_rows = [tiles[i] for i in range(1, num_rows, 2)]

odd_img_array = reconstruct_from_tile_rows(odd_tile_rows)
even_img_array = reconstruct_from_tile_rows(even_tile_rows)

odd_img = Image.fromarray(odd_img_array)
even_img = Image.fromarray(even_img_array)

display(odd_img)
display(even_img)

In [ ]:
def shred_image_by_cols(img_array, tile_w):
    """Shred an image array by odd/even columns of tile_w pixels each."""
    h, w = img_array.shape[:2]
    num_cols = w // tile_w

    cols = [img_array[:, j*tile_w:(j+1)*tile_w] for j in range(num_cols)]

    odd_cols = [cols[j] for j in range(0, num_cols, 2)]
    even_cols = [cols[j] for j in range(1, num_cols, 2)]

    odd_img = np.concatenate(odd_cols, axis=1)
    even_img = np.concatenate(even_cols, axis=1)
    return odd_img, even_img

odd_bycol_odd, odd_bycol_even = shred_image_by_cols(odd_img_array, tile_w)
even_bycol_odd, even_bycol_even = shred_image_by_cols(even_img_array, tile_w)

odd_odd_2_img = Image.fromarray(odd_bycol_odd)
odd_even_2_img = Image.fromarray(odd_bycol_even)
even_odd_2_img = Image.fromarray(even_bycol_odd)
even_even_2_img = Image.fromarray(even_bycol_even)

display(odd_odd_2_img)
display(odd_even_2_img)
display(even_odd_2_img)
display(even_even_2_img)


In [ ]:
standard_width = image.width

def resize_to_width(img, width):
    if img.width == width:
        return img
    new_height = int(img.height * width / img.width)
    return img.resize((width, new_height), Image.LANCZOS)

original_img = resize_to_width(image, standard_width)
odd_img = resize_to_width(odd_img, standard_width // 2)
even_img = resize_to_width(even_img, standard_width // 2)
odd_odd_img = resize_to_width(odd_odd_2_img, standard_width // 4)
odd_even_img = resize_to_width(odd_even_2_img, standard_width // 4)
even_odd_img = resize_to_width(even_odd_2_img, standard_width // 4)
even_even_img = resize_to_width(even_even_2_img, standard_width // 4)


In [ ]:
top_row = original_img

row2_height = max(odd_img.height, even_img.height)
row2 = Image.new('RGB', (standard_width, row2_height))
row2.paste(odd_img, (0, 0))
row2.paste(even_img, (odd_img.width, 0))

row3_height = max(odd_odd_img.height, odd_even_img.height, even_odd_img.height, even_even_img.height)
row3 = Image.new('RGB', (standard_width, row3_height))
row3.paste(odd_odd_img, (0, 0))
row3.paste(odd_even_img, (odd_odd_img.width, 0))
row3.paste(even_odd_img, (odd_odd_img.width + odd_even_img.width, 0))
row3.paste(even_even_img, (odd_odd_img.width + odd_even_img.width + even_odd_img.width, 0))

total_height = top_row.height + row2.height + row3.height
collage = Image.new('RGB', (standard_width, total_height), color='white')
y = 0
collage.paste(top_row, (0, y))
y += top_row.height
collage.paste(row2, (0, y))
y += row2.height
collage.paste(row3, (0, y))

display(collage)

In [ ]:
collage.save('collage.jpg')